In [1]:

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_fcbce0c99dd24dad845fde405c1a6547(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage V3 using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'dec963a04d9c4fe78ec0684a8e298a6f')
    hconf.set(prefix + '.username', 'e3beadc2523f4bbba7c9e59d6f4cfe95')
    hconf.set(prefix + '.password', 'D[t(E51_YwiZCIjm')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', True)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_fcbce0c99dd24dad845fde405c1a6547(name)

input_rdd = sc.textFile("swift://finalproject." + name + "/Ontime2013dfw.csv")
input_rdd.take(5)


[u'month,dated,WorkDay,unique_carrier,origin_city_name,dest_city_name,dephrs,arrhrs,schduration,distance,delay',
 u'1,2013-01-04,WEEKEND,UA,Chicago,Dallas/Fort,12,14,150,802,0',
 u'1,2013-01-04,WEEKEND,UA,Denver,Dallas/Fort,10,13,116,641,0',
 u'1,2013-01-04,WEEKEND,UA,Dallas/Fort,Chicago,13,16,136,802,0',
 u'1,2013-01-04,WEEKEND,UA,Newark,Dallas/Fort,6,9,247,1372,0']

In [3]:
dataheader= input_rdd.first()
dfw_data = input_rdd.filter(lambda x : x != dataheader)
dfw_data.take(5)

[u'1,2013-01-04,WEEKEND,UA,Chicago,Dallas/Fort,12,14,150,802,0',
 u'1,2013-01-04,WEEKEND,UA,Denver,Dallas/Fort,10,13,116,641,0',
 u'1,2013-01-04,WEEKEND,UA,Dallas/Fort,Chicago,13,16,136,802,0',
 u'1,2013-01-04,WEEKEND,UA,Newark,Dallas/Fort,6,9,247,1372,0',
 u'1,2013-01-04,WEEKEND,UA,Dallas/Fort,Denver,16,17,128,641,0']

In [111]:
def parse(line):
    fields = line.split(',')
    month = float(fields[0])
    workDay = fields[2]
    unique_carrier = fields[3]
    origin_city_name = fields[4]
    dest_city_name = fields[5]
    dephrs = float(fields[6])
    arrhrs = float(fields[7])
    schduration = float(fields[8])
    distance = float(fields[9])
    delay = float(fields[10])
    cls = ''
    if delay < 15:
        label = '0'
    else:
        label = '1'
    return  (month,unique_carrier,origin_city_name,dest_city_name,dephrs,arrhrs,schduration,distance,float(label))

In [112]:
dfw_rdd = dfw_data.map(parse)

In [113]:
dfw_rdd.take(5)

[(1.0, u'UA', u'Chicago', u'Dallas/Fort', 12.0, 14.0, 150.0, 802.0, 0.0),
 (1.0, u'UA', u'Denver', u'Dallas/Fort', 10.0, 13.0, 116.0, 641.0, 0.0),
 (1.0, u'UA', u'Dallas/Fort', u'Chicago', 13.0, 16.0, 136.0, 802.0, 0.0),
 (1.0, u'UA', u'Newark', u'Dallas/Fort', 6.0, 9.0, 247.0, 1372.0, 0.0),
 (1.0, u'UA', u'Dallas/Fort', u'Denver', 16.0, 17.0, 128.0, 641.0, 0.0)]

In [114]:
sqlContext=SQLContext(sc)


In [115]:
input_df =sqlContext.createDataFrame(dfw_rdd, ['month','unique_carrier','origin_city_name','dest_city_name', 'dephrs', 'arrhrs','schduration','distance','label'])

In [116]:
input_df.show()

+-----+--------------+----------------+--------------+------+------+-----------+--------+-----+
|month|unique_carrier|origin_city_name|dest_city_name|dephrs|arrhrs|schduration|distance|label|
+-----+--------------+----------------+--------------+------+------+-----------+--------+-----+
|  1.0|            UA|         Chicago|   Dallas/Fort|  12.0|  14.0|      150.0|   802.0|  0.0|
|  1.0|            UA|          Denver|   Dallas/Fort|  10.0|  13.0|      116.0|   641.0|  0.0|
|  1.0|            UA|     Dallas/Fort|       Chicago|  13.0|  16.0|      136.0|   802.0|  0.0|
|  1.0|            UA|          Newark|   Dallas/Fort|   6.0|   9.0|      247.0|  1372.0|  0.0|
|  1.0|            UA|     Dallas/Fort|        Denver|  16.0|  17.0|      128.0|   641.0|  0.0|
|  1.0|            UA|          Denver|   Dallas/Fort|  15.0|  18.0|      112.0|   641.0|  0.0|
|  1.0|            UA|     Dallas/Fort|    Washington|   7.0|  11.0|      171.0|  1172.0|  0.0|
|  1.0|            UA|      Washington| 

In [117]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

stringIndexer = StringIndexer(inputCol="unique_carrier", outputCol="unique_carrierIndex")
model = stringIndexer.fit(input_df)
indexed = model.transform(input_df)
encoder = OneHotEncoder(dropLast=False, inputCol="unique_carrierIndex", outputCol="unique_carrierVec")
encoded = encoder.transform(indexed)

stringIndexer1=StringIndexer(inputCol="origin_city_name", outputCol="origin_city_nameIndex")
model = stringIndexer1.fit(encoded)
indexed = model.transform(encoded)
encoder = OneHotEncoder(dropLast=False, inputCol="origin_city_nameIndex", outputCol="origin_city_nameVec")
encoded = encoder.transform(indexed)

stringIndexer2 = StringIndexer(inputCol="dest_city_name", outputCol="dest_city_nameIndex")
model = stringIndexer2.fit(encoded)
indexed = model.transform(encoded)
encoder = OneHotEncoder(dropLast=False, inputCol="dest_city_nameIndex", outputCol="dest_city_nameIndexVec")
encoded = encoder.transform(indexed)

transformeddf= encoded.select("month", "unique_carrierVec","origin_city_nameVec","dest_city_nameIndexVec","dephrs","arrhrs","schduration","distance","label")

In [118]:
transformeddf.show()

+-----+-----------------+-------------------+----------------------+------+------+-----------+--------+-----+
|month|unique_carrierVec|origin_city_nameVec|dest_city_nameIndexVec|dephrs|arrhrs|schduration|distance|label|
+-----+-----------------+-------------------+----------------------+------+------+-----------+--------+-----+
|  1.0|   (13,[6],[1.0])|    (127,[5],[1.0])|       (127,[0],[1.0])|  12.0|  14.0|      150.0|   802.0|  0.0|
|  1.0|   (13,[6],[1.0])|    (127,[7],[1.0])|       (127,[0],[1.0])|  10.0|  13.0|      116.0|   641.0|  0.0|
|  1.0|   (13,[6],[1.0])|    (127,[0],[1.0])|       (127,[5],[1.0])|  13.0|  16.0|      136.0|   802.0|  0.0|
|  1.0|   (13,[6],[1.0])|   (127,[23],[1.0])|       (127,[0],[1.0])|   6.0|   9.0|      247.0|  1372.0|  0.0|
|  1.0|   (13,[6],[1.0])|    (127,[0],[1.0])|       (127,[7],[1.0])|  16.0|  17.0|      128.0|   641.0|  0.0|
|  1.0|   (13,[6],[1.0])|    (127,[7],[1.0])|       (127,[0],[1.0])|  15.0|  18.0|      112.0|   641.0|  0.0|
|  1.0|   

In [119]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=["month", "unique_carrierVec", "origin_city_nameVec","dest_city_nameIndexVec","dephrs","arrhrs","schduration","distance"],
    outputCol="features")
output = assembler.transform(transformeddf)
print(output.select("features", "label").first())

Row(features=SparseVector(272, {0: 1.0, 7: 1.0, 19: 1.0, 141: 1.0, 268: 12.0, 269: 14.0, 270: 150.0, 271: 802.0}), label=0.0)


In [120]:
finaldf= output.select("features", "label")

In [121]:
finaldf.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(272,[0,7,19,141,...|  0.0|
|(272,[0,7,21,141,...|  0.0|
|(272,[0,7,14,146,...|  0.0|
|(272,[0,7,37,141,...|  0.0|
|(272,[0,7,14,148,...|  0.0|
|(272,[0,7,21,141,...|  0.0|
|(272,[0,7,14,149,...|  0.0|
|(272,[0,7,22,141,...|  0.0|
|(272,[0,7,15,141,...|  0.0|
|(272,[0,7,14,148,...|  1.0|
|(272,[0,7,14,166,...|  0.0|
|(272,[0,7,19,141,...|  0.0|
|(272,[0,7,19,141,...|  0.0|
|(272,[0,7,14,166,...|  0.0|
|(272,[0,7,16,141,...|  0.0|
|(272,[0,7,14,146,...|  0.0|
|(272,[0,7,14,148,...|  0.0|
|(272,[0,3,14,143,...|  0.0|
|(272,[0,3,19,141,...|  0.0|
|(272,[0,3,45,141,...|  0.0|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+----------+--------------------+--------------------+
|            features|label|prediction|       rawPrediction|         probability|
+--------------------+-----+----------+--------------------+--------------------+
|(272,[0,7,14,146,...|  0.0|       0.0|[1.27043438664910...|[0.78081709867488...|
|(272,[0,7,15,141,...|  0.0|       0.0|[1.60640105276880...|[0.83291112019251...|
|(272,[0,3,14,148,...|  0.0|       0.0|[1.23293374237094...|[0.77433163528321...|
|(272,[0,3,66,141,...|  0.0|       0.0|[1.07893743862191...|[0.74629285080078...|
|(272,[0,10,21,141...|  0.0|       0.0|[1.36279555244187...|[0.79621367258714...|
|(272,[0,10,14,148...|  0.0|       0.0|[1.15858364114330...|[0.76107525980476...|
|(272,[0,2,56,141,...|  0.0|       0.0|[0.99925317423100...|[0.73091171843632...|
|(272,[0,2,14,230,...|  0.0|       0.0|[1.07398784857199...|[0.74535455434056...|
|(272,[0,2,14,227,...|  0.0|       0.0|[0.72949095810343...|[0.67469355690360...|
|(272,[0,3,14,14

In [153]:
from pyspark.mllib.regression import LabeledPoint
import pyspark.sql.functions

labeledPoint= (finaldf.select(col("label"), col("features"))
  .map(lambda row: LabeledPoint(row.label, row.features)))

In [154]:
labeledPoint.take(5)

[LabeledPoint(0.0, (272,[0,7,19,141,268,269,270,271],[1.0,1.0,1.0,1.0,12.0,14.0,150.0,802.0])),
 LabeledPoint(0.0, (272,[0,7,21,141,268,269,270,271],[1.0,1.0,1.0,1.0,10.0,13.0,116.0,641.0])),
 LabeledPoint(0.0, (272,[0,7,14,146,268,269,270,271],[1.0,1.0,1.0,1.0,13.0,16.0,136.0,802.0])),
 LabeledPoint(0.0, (272,[0,7,37,141,268,269,270,271],[1.0,1.0,1.0,1.0,6.0,9.0,247.0,1372.0])),
 LabeledPoint(0.0, (272,[0,7,14,148,268,269,270,271],[1.0,1.0,1.0,1.0,16.0,17.0,128.0,641.0]))]

In [156]:
training, testing = labeledPoint.randomSplit([0.6, 0.4], seed=11)
training.take(5)



[LabeledPoint(0.0, (272,[0,7,19,141,268,269,270,271],[1.0,1.0,1.0,1.0,12.0,14.0,150.0,802.0])),
 LabeledPoint(0.0, (272,[0,7,21,141,268,269,270,271],[1.0,1.0,1.0,1.0,10.0,13.0,116.0,641.0])),
 LabeledPoint(0.0, (272,[0,7,14,146,268,269,270,271],[1.0,1.0,1.0,1.0,13.0,16.0,136.0,802.0])),
 LabeledPoint(0.0, (272,[0,7,21,141,268,269,270,271],[1.0,1.0,1.0,1.0,15.0,18.0,112.0,641.0])),
 LabeledPoint(0.0, (272,[0,7,14,166,268,269,270,271],[1.0,1.0,1.0,1.0,10.0,14.0,206.0,1372.0]))]

In [178]:
#Logistic regression with labeled points
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
model = LogisticRegressionWithLBFGS.train(training, numClasses=2)
predictionAndLabels = testing.map(lambda lp: (float(model.predict(lp.features)), lp.label))
from pyspark.mllib.evaluation import MulticlassMetrics
metrics = MulticlassMetrics(predictionAndLabels)
labels = labeledPoint.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s Accuracy = %s" % (label, metrics.precision(label)))
print("Weighted precision = %s" % metrics.weightedPrecision)
print("Weighted recall = %s" % metrics.weightedRecall)
print 'Confusion Matrix\n', metrics.confusionMatrix().toArray()

Class 0.0 Accuracy = 0.7833139845
Class 1.0 Accuracy = 0.315789473684
Weighted precision = 0.682004015475
Weighted recall = 0.783274171283
Confusion Matrix
[[  1.74755000e+05   1.30000000e+01]
 [  4.83420000e+04   6.00000000e+00]]
